In [ ]:
%matplotlib notebook
%pylab

<hr style="border-width:4px; border-color:coral; border-style:solid"/></br>

## Check accuracy

<hr style="border-width:4px; border-color:coral; border-style:solid"/>

This notebook shows you one way to check accuracy from your MPI runs.  In this example, we create files `trap_01.out`, `trap_02.out`, `trap_02.out` and so on for runs on 1, 2, 4, etc processors. You may need to modify th code so it works for your setup.

This code assumes that you are writing your output in binary.  The advantages of the binary output is that we store the full precision of the data in the smallest possible file size. 

See the notebook 'binary_output' for details on creating binary files. 

In [ ]:
# Range of processor counts
procs = [1,2,4,8]   # set to 1,2,4, and 8

# Range of N values
Nvec = [32,64,128,256,512,1024]

# Name of executable
exec_file = 'integral'

We'll use a Pandas DataFrame and a MultiIndex to store the data for all the runs.  This will allow us to easily compare errors across different processors.

The Pandas 'MultiIndex' is a nice feature to allow us to store multi-dimensional data.  Below, we store data for each tuple (P,N), where P is the processor count, and N is the dimension of the problem.   The data that we will store is the error.  

In [ ]:
import pandas

idx = pandas.IndexSlice

index = pandas.MultiIndex.from_product([procs,Nvec],names=['Proc','N'])
cols = ['Error']

df_error = pandas.DataFrame(index=index,columns = cols).sort_index()
index

We run all jobs above and collect the data in a Pandas data frame.   The errors should be identical (up to rounding error) for the same N, regardless of number of processors.  

In [ ]:
import subprocess
import shlex
import os

# Output file
filename = '{fexec:s}_{np:02}.out'.format

# mpirun command
shell_cmd = 'mpirun -n {np:d} {fexec:s} {N:d}'.format

dt = dtype([('N','int32'), ('Error','d')])      

for np in procs:
    output_fname = filename(fexec=exec_file,np=np)    
    try:
        os.remove(output_fname)
    except OSError as error: 
        #print(error)
        pass
    tvec = []
    for N in Nvec:
        cmd = shell_cmd(np=np,fexec=exec_file,N=N)
        arg_list = shlex.split(cmd) 
        output = subprocess.run(arg_list)
        
    fout = open(output_fname,"rb")
    data = fromfile(fout,dtype=dt)
    fout.close()

    df = pandas.DataFrame(data)
    df_error.loc[(np,),:] = df[cols].values
    
df_error.loc[idx[:],'Error'] = df_error.loc[idx[:],'Error'].astype('double')
df_error.unstack(1).style.format('{:.12e}'.format)

In [ ]:
figure(1)
clf()

# plot results from processor 1 (since they are all the same)
df_plot = df_error.loc[idx[1],:]

loglog(Nvec,df_plot.values,'.-',markersize=15,label='Time')

# Add slope to get best fit line
nv = array(Nvec).astype('d')
c = polyfit(log(nv),log(df_plot.values),1)
loglog(nv,exp(polyval(c,log(nv))),'r*-', markersize=8,\
         label='Best-fit line (slope={:6.2f})'.format(c[0][0]),linewidth=1)

# Add title, xlabel, ylabel, xticks and a legend
def fix_xticks(Nvec):
    p0 = log2(Nvec[0])
    p1 = log2(Nvec[-1])
    xlim([2**(p0-0.5), 2**(p1+0.5)])
    
    # Make nice tick marks
    pstr = (['{:d}'.format(int(N)) for N in Nvec])
    xticks(Nvec,pstr)

fix_xticks(Nvec)  # Need numpy array, not a Pandas 'Series'
xlabel("N",fontsize=16)
ylabel("Error",fontsize=16)
title("Errors in Left-endpoint method",fontsize=18)
legend()